# Historical tweet counts, by term

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_grid as altgrid

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Search for a term

In [5]:
term = "nigger"

#### Get daily counts

In [6]:
!twarc2 counts --granularity 'day' '{term}' --archive --csv data/raw/{term.replace(' ', '_')}.csv

100%|████████| Processed 16 years/16 years [03:44<00:00, 13106823 tweets total ]


In [7]:
src = pd.read_csv(f"data/raw/{term.replace(' ', '_')}.csv")

In [8]:
src.head()

,start,end,day_count
0,2022-10-06T00:00:00.000Z,2022-10-07T00:00:00.000Z,3143
1,2022-10-07T00:00:00.000Z,2022-10-08T00:00:00.000Z,10139
2,2022-10-08T00:00:00.000Z,2022-10-09T00:00:00.000Z,2378
3,2022-10-09T00:00:00.000Z,2022-10-10T00:00:00.000Z,1236
4,2022-10-10T00:00:00.000Z,2022-10-11T00:00:00.000Z,989


#### Clean up dates

In [9]:
src["year"] = pd.to_datetime(src["start"]).dt.strftime("%Y")
src["month_year"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m")
src["date"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m-%d")

In [10]:
src[src["date"] == src["date"].max()]

,start,end,day_count,year,month_year,date
30,2022-11-05T00:00:00.000Z,2022-11-05T02:01:33.000Z,337,2022,2022-11,2022-11-05


In [11]:
src_grouped = (
    src.groupby(["date", "year", "month_year"]).agg({"day_count": sum}).reset_index()
)

#### Slim down and re-order the dataframe

In [12]:
df = (
    src_grouped[["year", "month_year", "date", "day_count"]]
    .sort_values("date", ascending=False)
    .copy()
)

In [13]:
df.head()

,year,month_year,date,day_count
6073,2022,2022-11,2022-11-05,337
6072,2022,2022-11,2022-11-04,3573
6071,2022,2022-11,2022-11-03,1496
6070,2022,2022-11,2022-11-02,2854
6069,2022,2022-11,2022-11-01,5158


#### When was the first mention? 

In [14]:
first = df[df["day_count"] > 0]["date"].tail(1).iloc[0]
first

'2007-03-08'

#### How many total mentions? 

In [15]:
df.day_count.sum()

13106823

#### Average mentions? 

In [16]:
df.day_count.mean()

2157.8569311820875

#### Make a new dataframe starting from first mention

In [17]:
df_complete = df[df["date"] >= first]

#### Which day was mentioned the most? 

In [18]:
df_complete[df_complete["day_count"] == df_complete["day_count"].max()]

,year,month_year,date,day_count
5197,2020,2020-06,2020-06-12,57393


In [19]:
df_complete.sort_values("day_count", ascending=False).head(10)

,year,month_year,date,day_count
5197,2020,2020-06,2020-06-12,57393
4177,2017,2017-08,2017-08-27,55350
5074,2020,2020-02,2020-02-10,51847
5036,2020,2020-01,2020-01-03,45037
5097,2020,2020-03,2020-03-04,42349
4970,2019,2019-10,2019-10-29,41053
3952,2017,2017-01,2017-01-14,38666
4904,2019,2019-08,2019-08-24,38449
4660,2018,2018-12,2018-12-23,38317
5206,2020,2020-06,2020-06-21,36955


#### Chart it

In [20]:
alt.Chart(df_complete[df_complete["date"] > "2010-01-01"]).mark_area(
    color="red"
).encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(
    width=650, height=300, title=f"Daily mentions of '{term}' on Twitter"
)

alt.Chart(...)

---

## Aggregate 

#### Groupby month/year

In [21]:
months = (
    df_complete.groupby(["month_year"])
    .agg({"day_count": sum})
    .sort_values("month_year", ascending=False)
    .reset_index()
)

#### Chart it

In [22]:
alt.Chart(months.head(120)).mark_area().encode(
    x=alt.X("month_year:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=650, title=f"Monthly mentions of {term} on Twitter")

alt.Chart(...)

## Exports

In [23]:
clean_term = "n-word"

In [24]:
months.to_csv(
    f"data/processed/twitter_mentions_{clean_term.replace(' ', '')}_months.csv",
    index=False,
)
df_complete.to_csv(
    f"data/processed/twitter_mentions_{clean_term.replace(' ', '')}_days.csv",
    index=False,
)

In [25]:
months.head()

,month_year,day_count
0,2022-11,13418
1,2022-10,90274
2,2022-09,45097
3,2022-08,67547
4,2022-07,46113


In [26]:
months.day_count.mean()

69348.26984126984